In [33]:
import pandas as pd
import math
import numpy as np
import scipy.stats as stats
from scipy.stats import t

# Задание 1 для проекта по A/B - тестам.

### `chat_id ` из Телеграма:

In [2]:
chat_id = 42791670

------------------

<b>Контекст</b> <br>
Мы считаем, что наш продукт достаточно дорогой для клиента.<br>
<u><b>У нас есть гипотеза, что уменьшение цены продукта позволит повысить частоту продаж и суммарно увеличит доходность продукта.</b></u>
        
<b>Тест</b> <br>
Мы проводим тест, в рамках которого контролем является продажа продукта со старой ценой, а тестом — продажа продукта с новой ценой.
Целью этого теста является проверка гипотезы роста доходности нашего продукта из-за уменьшения тарифа.

<b>Статистические вводные</b> <br>
Уровень значимости 5%.
Мы хотели бы различать 5%-ное изменение целевой метрики с вероятностью 80%.

In [3]:
hist_data = pd.read_csv('hist_telesales.csv', index_col='ID')

In [4]:
hist_data

,Флаг дозвона,Флаг продажи,Расходы,PV,NPV
ID,,,,,
0,1,0,90,0,-90
1,0,0,5,0,-5
2,0,0,68,0,-68
3,1,0,22,0,-22
4,1,0,22,0,-22
...,...,...,...,...,...
72156,1,1,577,1346,769
72157,0,0,8,0,-8
72158,0,0,23,0,-23


In [5]:
# Считаем количество звонков и продаж
calls = hist_data['Флаг дозвона'].sum()
sales = hist_data['Флаг продажи'].sum()

# Рассчитываем конверсию звонков в продажи
conversion_rate = sales / calls

# Выводим результат
print('Конверсия звонков в продажи: {:.2%}'.format(conversion_rate))

Конверсия звонков в продажи: 50.37%


In [7]:
# мат ожидание
m1 = hist_data.NPV.mean() # среднне исторических данных
std1 = np.std(hist_data.NPV, ddof=1)

In [34]:
from statsmodels.stats.power import TTestIndPower

effect_size = 0.05 # ожидаемый эффект 5%
alpha = 0.05 # уровень значимости
power = 0.8 # мощность теста
ratio = 1.0 # отношение размера выборок

power_analysis = TTestIndPower()
sample_size = power_analysis.solve_power(effect_size=effect_size, alpha=alpha, power=power, ratio=ratio)
sample_size = math.ceil(sample_size)
print("Размер выборки: ", sample_size)

Размер выборки:  6281


In [ ]:
# Код выборки: 34346

In [19]:
df_cont = pd.read_csv('Контроль.csv') 

In [21]:
df_test = pd.read_csv('Тест.csv')

In [26]:
cont_mean = df_cont.NPV.mean() # мат ожидание по контрольной группе
test_mean = df_test.NPV.mean() # мат ожидание по тестовой группе

In [42]:
n1 = sample_size # размер контрольной выборкит 
n2 = sample_size # размер тестовой выборкит

In [43]:
s1 = df_cont.NPV.std() # стандаоьное оьклонние коньрольной группы
s2 = df_test.NPV.std() # стандаоьное оьклонние тестовой группы

In [44]:
sp = (((n1 - 1) * s1**2 + (n2 - 1) * s2**2) / (n1 + n2 - 2))**0.5  # объеденнное стандартное отклонение

In [56]:
t = (test_mean - cont_mean) / (sp * (1 / n1 + 1 / n2)**0.5)  # Т статистика

In [57]:
t_test = stats.t.sf(t, n1 + n2 - 2)

In [58]:
t_test

0.26598718963941326

Вывод:
- p-value не меньше чем уровень значимости alpha, значит принимаем H0 гипотезу.
- это значит что скидка в ночное время не позволила увеличить прибиль